## NOTES:

1) Why not first transfer reactions to cytoplasm and keep prokaryotic compartments and then run carveme's universe curation pipeline? We still would have the issue of futile cycles, which we could deactivate.

In [1]:
import cobra


uniprok = cobra.io.read_sbml_model("/home/robaina/Documents/NewAtlantis/phycosphere/data/carveme_universes/BIGG_universal_model/universal_model_cobrapy.xml")
uniprok

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


Name,bigg_universal
Memory address,7f84ec019bd0
Number of metabolites,15638
Number of reactions,28301
Number of genes,0
Number of groups,0
Objective expression,1.0*BIOMASS_reaction - 1.0*BIOMASS_reaction_reverse_5a818
Compartments,"cytoplasm, extracellular, periplasm, mitochondrion, peroxisome, unknown, nucleus, vacuole, golgi, thylakoid, lysosome, chloroplast, eyespot, flagellum, mitochondrial intermembrane space, unknown, unknown, unknown, unknown, mitochondrial membrane, cell wall, unknown"


## Preprocess universal model

Preprocess to remove non-prokaryotic compartments and transfer reactions to cytoplasm. This is to avoid CarveME's curation pipeline removing these reactions.

In [6]:
# from reconstruction import remove_shuttle_reactions, move_reactions_to_cytoplasm
import shutil
import gzip

import cobra
from cobra import Reaction


def gunzip_file(source_filepath, dest_filepath):
    with gzip.open(source_filepath, 'rb') as src_file:
        with open(dest_filepath, 'wb') as dest_file:
            shutil.copyfileobj(src_file, dest_file)


def remove_shuttle_reactions(
    model, allowed_compartments: set = {"c", "e", "p"}
):
    """
    Remove shuttle reactions between unwanted compartments.

    Args:
        model (Model): _description_
        allowed_compartments (set, optional): _description_. Defaults to {"c", "e", "p"}.

    Returns:
        Model: _description_
    """
    shuttle_rxns_in_unwanted_compartments = [
        rxn
        for rxn in model.reactions
        if (
            (len(rxn.compartments) > 1)
            and (not rxn.compartments.issubset(allowed_compartments))
        )
    ]
    model.remove_reactions(shuttle_rxns_in_unwanted_compartments, remove_orphans=True)
    return model


def move_reactions_to_cytoplasm(
    model, allowed_compartments: set = {"c", "e", "p"}
):
    """
    Update the metabolites of a reaction to include a new set of metabolites
    Args:
        reaction (Reaction): _description_
        allowed_compartments (set, optional): _description_. Defaults to {"c", "e", "p"}.

    Returns:
        Reaction: _description_
    """
    reactions_to_add = []
    reactions_to_remove = []
    for reaction in model.reactions:
        if not reaction.compartments.issubset(allowed_compartments):

            new_metabolites = {}
            for metabolite, stoich in reaction.metabolites.items():
                new_met_id = metabolite.id[:-1] + "c"
                new_metabolite = (
                    model.metabolites.get_by_id(new_met_id)
                    if new_met_id in model.metabolites
                    else metabolite.copy()
                )
                new_metabolite.compartment = "c"
                new_metabolite.id = new_met_id
                if new_met_id not in model.metabolites:
                    model.add_metabolites([new_metabolite])
                    model.remove_metabolites([metabolite])
                new_metabolites[new_metabolite] = stoich

            new_reaction = Reaction(
                id=reaction.id,
                name=reaction.name,
                lower_bound=reaction.lower_bound,
                upper_bound=reaction.upper_bound,
                subsystem=reaction.subsystem,
            )
            new_reaction.gene_reaction_rule = reaction.gene_reaction_rule
            new_reaction.add_metabolites(new_metabolites)
            reactions_to_add.append(new_reaction)
            reactions_to_remove.append(reaction)

    model.remove_reactions(reactions_to_remove, remove_orphans=True)
    model.add_reactions(reactions_to_add)
    return model

In [5]:
## Load universal BIGG model
# gunzip_file("carveme/data/generated/bigg_universe.xml.gz", "universal_bigg_model.xml")
# uniprok = cobra.io.read_sbml_model("universal_bigg_model.xml")
# uniprok

In [6]:
prok_compartments = {"e", "c", "p"}
uniprok = remove_shuttle_reactions(uniprok, allowed_compartments=prok_compartments)
uniprok = move_reactions_to_cytoplasm(uniprok)
uniprok

Name,bigg_universal
Memory address,7f84ec019bd0
Number of metabolites,11970
Number of reactions,25787
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"cytoplasm, extracellular, periplasm"


## Annotate metabolites

In [11]:
import pandas as pd
import re


def remove_compartment(met_id: str) -> str:
    return re.sub(r"_[a-z]$", "", met_id)


def annotate_compounds(model, cpd_annotations):
    """
    Annotate compounds in model: chemical formulae and charge

    Args:
        model (Model): _description_
        cpd_annotations (Path): _description_

    Returns:
        Model: _description_
    """
    cpd_db = pd.read_csv(cpd_annotations, sep="\t", index_col=0)
    for met in model.metabolites:
        met_id = remove_compartment(met.id)
        if met_id in cpd_db.index:
            met.formula = cpd_db.loc[met_id, "formula"]
            met.charge = cpd_db.loc[met_id, "charge"]
    return model

In [12]:
uniprok = cobra.io.read_sbml_model("universal_prokaryote.xml")

cpd_annotations = "/home/robaina/Documents/NewAtlantis/phycosphere/data/carveme_universes/mnx_compounds.tsv"
uniprok = annotate_compounds(uniprok, cpd_annotations)

## Change SBML flavor for compatibility

In [1]:
# from reframed import load_cbmodel, save_cbmodel, from_cobrapy

# # model = load_cbmodel(
# #     "/home/robaina/Documents/NewAtlantis/phycosphere/data/carveme_universes/BIGG_universal_model/universal_model_cobrapy.xml",
# #      flavor="fbc2"
# #      )

# model = from_cobrapy(uniprok)

# # save_cbmodel(model, "universal_model_bigg_flavor.xml", flavor="bigg")   ## this one doesn't exist but included in cofig?
# # save_cbmodel(model, "universal_model_cobra_flavor.xml", flavor="cobra")
# save_cbmodel(model, "universal_model_fbc2_flavor.xml", flavor="fbc2")

# # gunzip_file("carveme/data/generated/model_specific_data.csv.gz", "universal_model_bigg_flavor.csv")
# # gunzip_file("carveme/data/generated/model_specific_data.csv.gz", "universal_model_cobra_flavor.csv")
# gunzip_file("carveme/data/generated/model_specific_data.csv.gz", "universal_model_fbc2_flavor.csv")

In [ ]:
# from cobra.io import read_sbml_model
# from reframed import from_cobrapy, FBA

# cb_model = read_sbml_model("e_coli_core.xml")
# rf_model = from_cobrapy(cb_model)

## Write updated raw universal model

and perhaps generate the model specific data file?

In [13]:
cobra.io.write_sbml_model(uniprok, "universal_prokaryote.xml")
gunzip_file("carveme/data/generated/model_specific_data.csv.gz", "universal_prokaryote.csv")

In [14]:
from carveme.cli.curate_universe import curate


curate(
    inputfile="universal_prokaryote.xml",
    outputfile="prokaryote_carveme_curated.xml",
    taxa="bacteria",
    biomass="bacteria",
    biomass_db_path="carveme/data/input/biomass_db.tsv",
    normalize_biomass=False,
)

Curating bacteria universe...
Initial model size: 11970 x 25787
Computing missing formulae...
Removing unbalanced reactions..
found 5375 reactions
Current model size: 10395 x 19915
Trying to correct proton and charge balance...
Trying to fix hydrogen stoichiometry...
Removing blocked reactions and dead-end metabolites...
Final model size: 6861 x 17851
